# Tests of the binary module

## Jeff J. Andrews, Andreas Zezas, & Tassos Fragos

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import corner
import density_contour
import pickle
from astropy.coordinates import SkyCoord
from astropy import units as u

%matplotlib inline

sys.path.append('../binary')
import binary_evolve
import load_sse
sys.path.append('../stats')
import stats


In [ ]:
load_sse.load_sse()

In [ ]:
M1 = np.linspace(1.0, 39.0, 100)
M1_c = load_sse.func_sse_he_mass(M1)

x = 0.3
q_crit = (1.67 - x + 2*(M1_c/M1)**5) / 2.13
q_crit = 0.362 + 1.0 / (3.0*(1.0 - M1_c/M1))

#plt.plot(M1, M1_c)
plt.plot(M1, q_crit)
#plt.plot(M1, M1_c/M1)

#plt.ylim(0.0, 1.0)

plt.show()

In [ ]:
import matplotlib.patches as mpatches


def plot_star_evolve(mass_in):
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,6))

    ax1.set_title('Mass = ' + str(mass_in))
    
    # Set x values
    t_tmp = np.linspace(1.0, 100.0, 100000)

    # Get y values
    mass_tmp, mdot_tmp, radius_tmp, k_tmp = load_sse.func_get_sse_star(mass_in, t_tmp)

    # Get times for k-type change
    _, idx = np.unique(k_tmp, return_index=True)

    # Set patches, colors, etc
    patches = np.array([])
    colors=['k','k','g','r','b','c','y','m','k']


    # Plot k-types
#    ax1.plot(t_tmp, k_tmp)
#    ax1.set_ylim(0.0, 15.0)
#    for i in np.arange(len(t_tmp[np.sort(idx)])-1):
#        t = t_tmp[np.sort(idx)][i+1]
#        ax1.axvline(t, color='k', linestyle='--')

    
    # Plot stellar radii - linear
    ax1.plot(t_tmp, radius_tmp)
    for i in np.arange(len(t_tmp[np.sort(idx)])-2):

        t = t_tmp[np.sort(idx)][i+2]
        t_last = t_tmp[np.sort(idx)][i+1]
    
        # Get k-type
        k = int(k_tmp[np.sort(idx)][i+1])

        # Fill in with different colors
        ax1.fill_between(t_tmp, 0.0, radius_tmp, where=((t_tmp<t) & (t_tmp>t_last)), facecolor=colors[k], interpolate=True, alpha=0.5)

        # Hatched regions designate stars with convective envelopes
        if (k==3) or (k==5) or (k==6) or (k==8) or (k==9):
            ax1.fill_between(t_tmp, 0.0, radius_tmp, where=((t_tmp<t) & (t_tmp>t_last)), color="none", hatch='//', edgecolor='k', interpolate=True)
    
        k_str = str(k)

        
        patches = np.append(patches, mpatches.Patch(color=colors[k], label=k_str))

        
        
    # Plot stellar radii - log scale
    ax2.plot(t_tmp, radius_tmp)
    for i in np.arange(len(t_tmp[np.sort(idx)])-2):

        t = t_tmp[np.sort(idx)][i+2]
        t_last = t_tmp[np.sort(idx)][i+1]

        # Get k-type
        k = int(k_tmp[np.sort(idx)][i+1])

        # Fill in with different colors
        ax2.fill_between(t_tmp, 0.0, radius_tmp, where=((t_tmp<t) & (t_tmp>t_last)), facecolor=colors[k], interpolate=True, alpha=0.5)

        # Hatched regions designate stars with convective envelopes
        if (k==3) or (k==5) or (k==6) or (k==8) or k==9:
            ax2.fill_between(t_tmp, 0.0, radius_tmp, where=((t_tmp<t) & (t_tmp>t_last)), color="none", hatch='//', edgecolor='k', interpolate=True)
    
        k_str = str(k)
    
#        patches = np.append(patches, mpatches.Patch(color=colors[i], label=k_str))


    # Create lower panel legend
    ax1.legend(title='k-type', handles=patches.tolist(), loc=2)

    # Set labels, limits
    delta_x = t_tmp[idx][-1] - t_tmp[idx][1]
    xlim = [- 0.15*delta_x + t_tmp[idx][1], 0.05*delta_x + t_tmp[idx][-1]]
    ax1.set_xlim(xlim)
    ax2.set_xlim(xlim)
    ylim = [1.0, 2.0*max(radius_tmp)]
    ax2.set_ylim(ylim)

    ax1.set_ylabel('k-type')
    ax2.set_ylabel('Radius (Rsun)')
    ax2.set_xlabel('Time (Myr)')
    ax2.set_yscale('log')

    return plt
    


In [ ]:
plot_star_evolve(8.0)
plt.savefig('../figures/stellar_type_evolve_M8.pdf')

plot_star_evolve(10.0)
plt.savefig('../figures/stellar_type_evolve_M10.pdf')

plot_star_evolve(14.0)
plt.savefig('../figures/stellar_type_evolve_M14.pdf')

plot_star_evolve(30.0)
plt.savefig('../figures/stellar_type_evolve_M30.pdf')

#plt.show()

In [ ]:
def plot_k_r(mass_in):

    
    # Set x values
    t_tmp = np.linspace(1.0, 100.0, 100000)

    # Get y values
    mass_tmp, mdot_tmp, radius_tmp, k_tmp = load_sse.func_get_sse_star(mass_in, t_tmp)

    # Get times for k-type change
    _, idx = np.unique(k_tmp, return_index=True)

    # Set patches, colors, etc
    patches = np.array([])
    colors=['k','k','g','r','b','c','y','m','k']

    
    tmp_r = np.linspace(1.0, 1800.0, 10000)
    tmp_k = load_sse.func_sse_get_k_from_r(mass_in, tmp_r)
    
    plt.plot(tmp_r, tmp_k)
    
    
#     # Plot k-types from time
#     plt.plot(t_tmp, k_tmp)
#     plt.ylim(0.0, 15.0)
#     for i in np.arange(len(t_tmp[np.sort(idx)])-1):
#         t = t_tmp[np.sort(idx)][i+1]
#         plt.axvline(t, color='k', linestyle='--')
        
        
    plt.show()


In [ ]:
plot_k_r(10.0)

In [ ]:
plot_star_evolve(10.0)

plt.show()